# Sentiment Analysis with Deep learning using BERT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# EDA and Preprocessing

In [2]:
import torch
import pandas as pd

In [3]:
from tqdm.notebook import tqdm

In [4]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Sentiment analysis with BERT/Data/smile-annotations-final.csv', names=['id', 'text', 'category'])
df.set_index('id', inplace=True)

In [5]:
df.head()

,text,category
id,,
NaN,"pel @mitpress @monoskop http://t.co/fxp5kjuGV2""",nocode
6.121680e+17,@stiveshouse @MillenniumArt @Tate_StIves @Trem...,happy
6.116340e+17,I miss Cornwall so much. #ff @FernPitCafe @Pen...,not-relevant
6.107630e+17,£10 off @accessart http://t.co/0biFQaXTQg memb...,not-relevant
6.115290e+17,@britishmuseum @Ophiolatrist Spain? Egypt?,nocode


In [6]:
df.text.iloc[0]

'pel @mitpress @monoskop http://t.co/fxp5kjuGV2"'

In [7]:
df = df.dropna()

In [8]:
df.category.value_counts()

nocode               1555
happy                1123
not-relevant          211
angry                  56
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [9]:
df = df[~df.category.str.contains('\|')]

In [10]:
df = df[df.category != 'nocode']

In [11]:
df.category.value_counts()

happy           1123
not-relevant     211
angry             56
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [12]:
possible_labels = df.category.unique()

In [13]:
label_dict = {}
for i, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = i

In [14]:
label_dict

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [15]:
df['label'] = df.category.replace(label_dict)
df.head(15)

,text,category,label
id,,,
6.121680e+17,@stiveshouse @MillenniumArt @Tate_StIves @Trem...,happy,0
6.116340e+17,I miss Cornwall so much. #ff @FernPitCafe @Pen...,not-relevant,1
6.107630e+17,£10 off @accessart http://t.co/0biFQaXTQg memb...,not-relevant,1
6.151070e+17,Enjoying the amazing collections @britishmuseu...,happy,0
6.112390e+17,@NationalGallery Beautiful painting. Thank you.,happy,0
6.125050e+17,#definingbeauty @britishmuseum a truly breatht...,happy,0
6.149110e+17,"@laiyaharvey thank you for the favourite, foll...",happy,0
6.125400e+17,Treats for the end of June @HeatonsArtTrail fo...,happy,0
6.115830e+17,@tatteredstones @kettlesyard One of the best p...,happy,0


# Training/Validation Split

In [61]:
from sklearn.model_selection import train_test_split

In [62]:
X_train, X_test, y_train, y_test = train_test_split(
    df.index.values,
    df.label.values,
    test_size = 0.15,
    random_state=17,
    stratify=df.label.values
)

In [63]:
df['data_type'] = ['not_set']*df.shape[0]

In [64]:
df.head()

,text,category,label,data_type
id,,,,
6.121680e+17,@stiveshouse @MillenniumArt @Tate_StIves @Trem...,happy,0,not_set
6.116340e+17,I miss Cornwall so much. #ff @FernPitCafe @Pen...,not-relevant,1,not_set
6.107630e+17,£10 off @accessart http://t.co/0biFQaXTQg memb...,not-relevant,1,not_set
6.151070e+17,Enjoying the amazing collections @britishmuseu...,happy,0,not_set
6.112390e+17,@NationalGallery Beautiful painting. Thank you.,happy,0,not_set


In [65]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_test, 'data_type'] = 'test'

In [66]:
df.groupby(['category', 'label', 'data_type']).count()

text
category     label data_type      
angry        2     test         15
                   train        41
disgust      3     test          2
                   train         4
happy        0     test        251
                   train       872
not-relevant 1     test         51
                   train       160
sad          4     test          8
                   train        24
surprise     5     test         11
                   train        24

# Loading Tokenizer and Encoding Data

In [67]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [68]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

In [69]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=256,
    return_tensors='pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    df[df.data_type=='test'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df[df.data_type=='test'].label.values)


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [70]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [71]:
len(dataset_train)

1125

In [72]:
len(dataset_test)

338

# Setting BERT Pretrained Model

In [73]:
from transformers import BertForSequenceClassification

In [74]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(label_dict),
    output_attentions=False,
    output_hidden_states=False        
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# Creating Data Loaders

In [75]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [76]:
batch_size = 32

dataloader_train = DataLoader(
    dataset_train,
    sampler = RandomSampler(dataset_train),
    batch_size=batch_size
)
dataloader_test = DataLoader(
    dataset_test,
    sampler = RandomSampler(dataset_test),
    batch_size=32
)

# Setting up Optimizer and Schedular

In [77]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [102]:
optimizer = AdamW(
    model.parameters(),
    lr = 5e-5,
    eps = 1e-8
)

In [103]:
epochs = 10
schedular = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = len(dataloader_train)*epochs
)

# Defining Performance Metrics

In [104]:
import numpy as np
from sklearn.metrics import f1_score

In [105]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [106]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k,v in label_dict.items()}
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

# Creating Training Loop

In [107]:
import random
 
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [108]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [109]:
def evaluate(dataloader_test):
    
    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2],
                 }
        
        with torch.no_grad():
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()
        
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
        
    loss_val_avg = loss_val_total/len(dataloader_test)
    
    predictions = np.concatenate(predictions, axis = 0)
    true_vals = np.concatenate(true_vals, axis = 0)
    
    return loss_val_avg, predictions, true_vals

In [110]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train,
                        desc = 'Epoch{:1d}'.format(epoch),
                        leave=False,
                        disable=False)
    
    
    for batch in progress_bar:
        
        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids'      : batch[0],
            'attention_mask': batch[1],
            'labels'         : batch[2]
        }
        
        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        schedular.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
    
    torch.save(model.state_dict(), f'Model_BERT_ft_epoch{epoch}.model')
    
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write('Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_test)
    
    f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {f1}')


Epoch {epoch}
Training loss: {loss_train_avg}
Validation loss: 0.6192285228859294
F1 Score (weighted): 0.7577408931830213



Epoch {epoch}
Training loss: {loss_train_avg}
Validation loss: 0.720458792691881
F1 Score (weighted): 0.7834432683397364



Epoch {epoch}
Training loss: {loss_train_avg}
Validation loss: 0.4248941550877961
F1 Score (weighted): 0.881286109619199



Epoch {epoch}
Training loss: {loss_train_avg}
Validation loss: 0.5970695045861331
F1 Score (weighted): 0.8634321732832131



Epoch {epoch}
Training loss: {loss_train_avg}
Validation loss: 0.6079175675457175
F1 Score (weighted): 0.8555957421340784



Epoch {epoch}
Training loss: {loss_train_avg}
Validation loss: 0.6116521968082949
F1 Score (weighted): 0.8667575840754395



Epoch {epoch}
Training loss: {loss_train_avg}
Validation loss: 0.6984803757884286
F1 Score (weighted): 0.8613967213135427



Epoch {epoch}
Training loss: {loss_train_avg}
Validation loss: 0.6720555180853064
F1 Score (weighted): 0.8678302558126512



Epoch {epoch}
Training loss: {loss_train_avg}
Validation loss: 0.6614146300337531
F1 Score (weighted): 0.8820192868459004



Epoch {epoch}
Training loss: {loss_train_avg}
Validation loss: 0.6738681684840809
F1 Score (weighted): 0.8685979877703901



# Loading and Evaluating Model

In [111]:
model  = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                       num_labels=len(label_dict),
                                                       output_attentions=False,
                                                       output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [112]:
model.to(device)
pass

In [119]:
model.load_state_dict(torch.load('Model_BERT_ft_epoch10.model',
                                 map_location=torch.device('cpu')))

<All keys matched successfully>

In [120]:
_, predictions, true_vals = evaluate(dataloader_test)

In [121]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 244/251

Class: not-relevant
Accuracy: 33/51

Class: angry
Accuracy: 12/15

Class: disgust
Accuracy: 0/2

Class: sad
Accuracy: 1/8

Class: surprise
Accuracy: 7/11

